# Baseline

In [1]:
import numpy as np
import pandas as pd
# from lightgbm import LGBMClassifier
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
from sklearn.preprocessing import LabelEncoder
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import copy
import math
import pickle
import random
from sklearn.metrics import roc_auc_score
import optuna
import catboost as cb
import warnings
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
warnings.filterwarnings('ignore')
from tqdm.auto import tqdm

import multiclass_boosting
from multiclass_boosting import Boosting

## Загрузка данных

In [4]:
train_df = pd.read_parquet("train_data.pqt")
test_df = pd.read_parquet("test_data.pqt")

вытягиваем в колбасу по id

In [6]:
grouped_df_first = test_df.groupby('id').first().reset_index()
merged_df = pd.merge(test_df, grouped_df_first, on='id', suffixes=('', '_first'))

grouped_df_second = test_df.groupby('id').nth(1).reset_index()
merged_df = pd.merge(merged_df, grouped_df_second, on='id', suffixes=('', '_second'))

merged_df = merged_df.loc[:, ~merged_df.columns.duplicated()]

In [7]:
test_df = merged_df[merged_df['date'] == 'month_6']

In [8]:
grouped_df_first = train_df.groupby('id').first().reset_index()
merged_df = pd.merge(train_df, grouped_df_first, on='id', suffixes=('', '_first'))

grouped_df_second = train_df.groupby('id').nth(1).reset_index()
merged_df = pd.merge(merged_df, grouped_df_second, on='id', suffixes=('', '_second'))

merged_df = merged_df.loc[:, ~merged_df.columns.duplicated()]

In [9]:
train_df = merged_df[merged_df['date'] == 'month_3']

удаляем сильную корреляцию

In [ ]:
corr_matrix = train_df.corr().abs()

# Получение верхнего треугольника матрицы корреляции (без диагонали)
upper_triangle = corr_matrix.where(
    np.triu(np.ones(corr_matrix.shape), k=1).astype(np.bool))

# Нахождение колонок, где корреляция больше 0.95
to_drop = [column for column in upper_triangle.columns if any(upper_triangle[column] > 0.95)]

In [12]:
test_df = test_df.drop(to_drop, axis=1)
train_df = train_df.drop(to_drop, axis=1)

кодируем кластера

In [13]:
label_encoder = LabelEncoder()
train_df['start_cluster'] = label_encoder.fit_transform(train_df['start_cluster'])
train_df['end_cluster'] = label_encoder.transform(train_df['end_cluster'])

train_df['start_cluster_first'] = label_encoder.fit_transform(train_df['start_cluster_first'])
train_df['start_cluster_second'] = label_encoder.fit_transform(train_df['start_cluster_second'])

train_df['end_cluster_first'] = label_encoder.transform(train_df['end_cluster_first'])
train_df['end_cluster_second'] = label_encoder.transform(train_df['end_cluster_second'])

test_df['start_cluster'] = label_encoder.fit_transform(test_df['start_cluster'])
test_df['start_cluster_first'] = label_encoder.fit_transform(test_df['start_cluster_first'])
test_df['start_cluster_second'] = label_encoder.fit_transform(test_df['start_cluster_second'])

category_mapping = dict(zip(label_encoder.classes_, label_encoder.transform(label_encoder.classes_)))

правильный тип данных для признаков

In [18]:
cat_cols = [
    "channel_code", "city", "city_type",
    "okved", "segment", "start_cluster",
    "index_city_code", "ogrn_month", "ogrn_year",
]

modified_cat_cols = cat_cols + [item + '_first' for item in cat_cols] + [item + '_second' for item in cat_cols]

print(modified_cat_cols)

['channel_code', 'city', 'city_type', 'okved', 'segment', 'start_cluster', 'index_city_code', 'ogrn_month', 'ogrn_year', 'channel_code_first', 'city_first', 'city_type_first', 'okved_first', 'segment_first', 'start_cluster_first', 'index_city_code_first', 'ogrn_month_first', 'ogrn_year_first', 'channel_code_second', 'city_second', 'city_type_second', 'okved_second', 'segment_second', 'start_cluster_second', 'index_city_code_second', 'ogrn_month_second', 'ogrn_year_second']


In [19]:
cat_cols = modified_cat_cols

for cat in cat_cols:
    train_df[cat] = train_df[cat].fillna('missing')
    test_df[cat] = test_df[cat].fillna('missing')

train_df[cat_cols] = train_df[cat_cols].astype("str")
test_df[cat_cols] = test_df[cat_cols].astype("str")

train_df['end_cluster'] = train_df['end_cluster'].astype('str')

In [20]:
features = list(set(train_df.drop(["id", "date", "end_cluster", "end_cluster_first", "end_cluster_second", "date_first", "date_second"], axis=1).columns))

In [37]:
len(features)

136

In [22]:
y = train_df['end_cluster']
X = train_df[features]


X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.1, random_state=55)

In [38]:
features 

['ogrn_month_second',
 'sum_b_oper_1m_first',
 'sum_deb_d_oper_1m',
 'sum_deb_e_oper_3m_first',
 'cnt_days_cred_e_oper_1m',
 'sum_cred_h_oper_1m_second',
 'sum_cred_h_oper_1m',
 'sum_deb_g_oper_1m',
 'sum_of_paym_2m',
 'cnt_a_oper_1m_second',
 'sum_deb_f_oper_1m_second',
 'sum_deb_d_oper_1m_second',
 'sum_cred_f_oper_3m',
 'sum_cred_f_oper_1m_first',
 'sum_cred_h_oper_3m',
 'sum_cred_h_oper_1m_first',
 'sum_deb_f_oper_1m',
 'sum_a_oper_1m_second',
 'balance_amt_avg_second',
 'index_city_code_second',
 'segment_first',
 'ogrn_month',
 'cnt_c_oper_1m',
 'cnt_days_deb_f_oper_1m_second',
 'start_cluster_first',
 'cnt_days_cred_g_oper_1m',
 'city',
 'sum_deb_h_oper_1m',
 'sum_of_paym_2m_second',
 'balance_amt_avg_first',
 'city_type',
 'sum_c_oper_1m',
 'cnt_deb_d_oper_1m',
 'cnt_cred_h_oper_1m',
 'city_first',
 'start_cluster',
 'sum_cred_h_oper_3m_first',
 'sum_a_oper_3m_first',
 'sum_of_paym_2m_first',
 'cnt_days_cred_f_oper_3m_first',
 'sum_deb_f_oper_3m',
 'sum_deb_e_oper_1m_first',
 '

треним модель

In [26]:
boosting = Boosting(X_train, X_val, y_train, y_val, cat_features = modified_cat_cols)

Init...
Init Finished!


In [37]:
boosting.train()

0:	test: 0.8249259	best: 0.8249259 (0)	total: 8.97s	remaining: 44m 43s
100:	test: 0.9556834	best: 0.9556834 (100)	total: 19m 13s	remaining: 37m 52s
200:	test: 0.9553025	best: 0.9560956 (102)	total: 49m 47s	remaining: 24m 31s
Stopped by overfitting detector  (100 iterations wait)

bestTest = 0.9560956249
bestIteration = 102

Shrink model to first 103 iterations.


сохраняем загружаем

In [38]:
boosting.save_model('model_new4.pkl')

In [33]:
boosting.load_model('model_new4.pkl')

подставляем значения для начального кластера 6 месяца из модели Алины

In [46]:
test_alina = pd.read_parquet('test_df.pqt')

In [49]:
test_alina[test_alina['date']=='month_6'][['id', 'start_cluster']]

,id,start_cluster
2,200000,{α}
5,200001,{α}
8,200002,{other}
11,200003,{α}
13,200004,{α}
...,...,...
290108,299995,{α}
290111,299996,{α}
290114,299997,{α}
290116,299998,{α}


In [52]:
test_df['start_cluster'] = test_alina[test_alina['date']=='month_6']['start_cluster']
test_df['start_cluster'] = test_df['start_cluster'].map(category_mapping)

In [58]:
test_df

,id,date,balance_amt_avg,balance_amt_max,balance_amt_min,channel_code,city,city_type,index_city_code,ogrn_days_end_month,...,sum_deb_e_oper_1m_second,sum_deb_f_oper_1m_second,cnt_days_deb_f_oper_1m_second,sum_cred_f_oper_1m_second,sum_deb_h_oper_1m_second,sum_cred_h_oper_1m_second,sum_deb_f_oper_3m_second,sum_cred_f_oper_3m_second,sum_cred_h_oper_3m_second,start_cluster_second
2,200000,month_6,0.045988,0.049418,-0.125995,channel_code_12,city_14,city_type_0,None,-1.533705,...,0.119735,1.399146,0.244786,-0.017983,-0.145537,0.363179,3.136964,-0.025646,0.237817,15
5,200001,month_6,-0.156722,-0.204920,-0.125856,channel_code_9,city_76,city_type_0,None,0.092087,...,-0.142792,-0.173564,NaN,-0.017983,-0.158938,-0.171825,-0.213518,-0.025646,-0.201123,15
8,200002,month_6,-0.048015,0.448252,-0.125995,channel_code_12,city_14,city_type_0,index_city_code_78,-1.069193,...,2.786193,0.231784,0.309302,-0.017983,5.075587,7.251601,0.567790,-0.025646,3.313686,0
11,200003,month_6,-0.156579,-0.204813,-0.125501,channel_code_7,city_31,city_type_0,None,-0.256297,...,-0.142792,-0.173564,NaN,-0.017983,-0.158938,-0.171825,-0.213518,-0.025646,-0.201123,15
13,200004,month_6,-0.153379,-0.201932,-0.125995,channel_code_7,None,None,None,0.672727,...,-0.142470,-0.173564,0.180270,-0.017983,-0.158173,-0.171825,-0.213518,-0.025646,-0.201123,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
290108,299995,month_6,-0.153707,-0.202806,-0.125498,channel_code_14,city_22,city_type_0,index_city_code_29,1.601751,...,-0.142792,-0.173564,NaN,-0.017983,-0.158938,-0.171825,-0.213518,-0.025646,-0.201123,15
290111,299996,month_6,-0.154929,-0.197878,-0.125873,channel_code_1,city_96,city_type_0,index_city_code_66,-0.604681,...,-0.141009,-0.168077,0.212528,-0.017983,-0.155710,-0.167223,-0.209554,-0.025646,-0.199132,15
290114,299997,month_6,-0.105294,-0.141429,-0.104590,channel_code_17,city_85,city_type_0,index_city_code_103,0.440471,...,-0.026755,-0.166493,0.212528,-0.017983,0.106586,0.082892,-0.198552,-0.025646,0.070113,15
290116,299998,month_6,-0.155350,-0.203711,-0.125995,channel_code_9,city_25,city_type_0,index_city_code_30,0.672727,...,-0.142792,-0.173564,0.180270,-0.017983,-0.158938,-0.171825,-0.213518,-0.025646,-0.201123,1


In [59]:
test = test_df[test_df['date'] == 'month_6']

получаем ответ на тесте

In [61]:
ans = boosting.model.predict_proba(test[features])

In [62]:
data = {} #{'id': test.id.to_list()}
for cls, prob in zip(category_mapping.keys(), ans):
    data[cls] = prob

column_mapping = {
    0: '{other}',
    1: '{}',
    2: '{α, β}',
    3: '{α, γ}',
    4: '{α, δ}',
    5: '{α, ε, η}',
    6: '{α, ε, θ}',
    7: '{α, ε, ψ}',
    8: '{α, ε}',
    9: '{α, η}',
    10: '{α, θ}',
    11: '{α, λ}',
    12: '{α, μ}',
    13: '{α, π}',
    14: '{α, ψ}',
    15: '{α}',
    16: '{λ}'
}

output = pd.DataFrame(ans)
output = output.rename(columns=column_mapping)
output = output.assign(id=test['id'].tolist())

sample_submission_df = pd.read_csv("sample_submission.csv")
output[list(sample_submission_df.columns)].to_csv('ans_13_10_04.csv', index=False)

In [63]:
output[list(sample_submission_df.columns)]

,id,{other},{},"{α, β}","{α, γ}","{α, δ}","{α, ε, η}","{α, ε, θ}","{α, ε, ψ}","{α, ε}","{α, η}","{α, θ}","{α, λ}","{α, μ}","{α, π}","{α, ψ}",{α},{λ}
0,200000,0.026027,0.006934,0.015620,0.042491,0.013467,0.000212,0.001979,0.000474,0.012681,0.008145,0.012790,0.001654,0.003826,0.000017,0.011713,0.841938,0.000033
1,200001,0.024784,0.398789,0.002742,0.005277,0.000912,0.000211,0.000389,0.000119,0.002777,0.013073,0.001753,0.000465,0.001281,0.000006,0.001375,0.545750,0.000297
2,200002,0.734431,0.001664,0.007622,0.073085,0.015237,0.000792,0.001564,0.011077,0.021906,0.004587,0.003121,0.005196,0.003607,0.000059,0.058196,0.057765,0.000091
3,200003,0.045788,0.439252,0.001696,0.003828,0.000976,0.000279,0.000180,0.000064,0.001912,0.022473,0.001416,0.000138,0.001479,0.000004,0.000642,0.479800,0.000074
4,200004,0.091250,0.096475,0.009334,0.017615,0.003444,0.000771,0.000345,0.000207,0.006090,0.091952,0.002381,0.000593,0.004388,0.000006,0.001092,0.673916,0.000142
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
99995,299995,0.014302,0.202462,0.004281,0.007603,0.003083,0.000034,0.000160,0.000082,0.001424,0.000948,0.000774,0.000986,0.000373,0.000007,0.001195,0.762018,0.000267
99996,299996,0.035458,0.015601,0.022608,0.067935,0.017685,0.000084,0.000455,0.000321,0.010691,0.003285,0.004189,0.007985,0.001549,0.000012,0.003999,0.807636,0.000509
99997,299997,0.046212,0.013424,0.030571,0.092887,0.017133,0.000066,0.000553,0.000478,0.011875,0.003699,0.003330,0.001795,0.003235,0.000006,0.014739,0.759970,0.000027
99998,299998,0.062943,0.143641,0.015112,0.066738,0.009668,0.000381,0.000827,0.000424,0.013521,0.011664,0.003417,0.003860,0.001081,0.000019,0.004017,0.661756,0.000929
